In [1]:
%load_ext autoreload
%load_ext autotime

time: 332 µs (started: 2023-01-25 21:34:23 -08:00)


In [2]:
import sys
sys.path.append("../")
import warnings
warnings.filterwarnings('ignore')
from bokeh.io import output_notebook
output_notebook()

Loading BokehJS ...

time: 1.58 s (started: 2023-01-25 21:34:23 -08:00)


In [3]:
import dask.dataframe as dd
import numpy as np
import pandas as pd


time: 661 ms (started: 2023-01-25 21:34:27 -08:00)


In [4]:
%autoreload 2

from vani.recorder import RecorderAnalyzer

# log_dir = "/p/vast1/iopp/recorder_app_logs/cm1/nodes-32/workflow-4/_parquet"
log_dir = "/p/vast1/iopp/recorder_app_logs/hacc/nodes-32/workflow-0/_parquet"
# log_dir = "/p/gpfs1/iopp/recorder_app_logs/montage/nodes-32/_parquet"
# log_dir = "/p/gpfs1/iopp/recorder_app_logs/genome_pegasus/nodes-32/_parquet"
# log_dir = "/p/gpfs1/iopp/recorder_app_logs/lbann-jag/nodes-32/_parquet"

recorder_analyzer = RecorderAnalyzer(
    cluster_manager_args=dict(
        force_local=True,
        cluster_settings=dict(
            cores=4,
            dashboard_port=3656,
            local_directory="/var/tmp/dask-recorder",
            log_file="%J.log",
            worker_queue="pdebug"
        )
    ),
    working_dir='.recorder',
    debug=True
) 

recorder_analyzer

time: 2.5 s (started: 2023-01-25 21:34:29 -08:00)


In [5]:
recorder_analyzer.cluster_manager.clients

[<Client: 'tcp://192.168.132.138:43271' processes=4 threads=36>]

time: 3.53 ms (started: 2023-01-25 21:34:39 -08:00)


In [6]:
%autoreload 2

res = recorder_analyzer.analyze_parquet(log_dir=log_dir)

view trange 29
view file_id 1231
view proc_id 1280
view trange subview trange 29 29
view trange subview file_id 29 1231
view trange subview proc_id 29 1280
view file_id subview trange 1231 29
view file_id subview file_id 1231 1231
view file_id subview proc_id 1231 1280
view proc_id subview trange 1280 29
view proc_id subview file_id 1280 1231
view proc_id subview proc_id 1280 1280
view trange subview trange llc trange 29 29 32987
view trange subview trange llc file_id 29 29 31011
view trange subview trange llc proc_id 29 29 34224
view trange subview file_id llc trange 29 1231 32987
view trange subview file_id llc file_id 29 1231 31011
view trange subview file_id llc proc_id 29 1231 34224
view trange subview proc_id llc trange 29 1280 32987
view trange subview proc_id llc file_id 29 1280 31011
view trange subview proc_id llc proc_id 29 1280 34224
view file_id subview trange llc trange 1231 29 32987
view file_id subview trange llc file_id 1231 29 31011
view file_id subview trange llc pro

In [7]:
views, subviews, llcviews = res

time: 788 µs (started: 2023-01-25 21:35:10 -08:00)


In [8]:
llcviews[('trange', 'file_id', 'proc_id')] \
    .compute() \
    .set_index(['trange', 'file_id', 'proc_id', 'io_cat']) \
    .sort_values(('duration', 'sum'), ascending=False)


acc_pat      duration  \
                                                          min max       sum   
trange file_id             proc_id             io_cat                         
19     6142509188972423790 5117778030855898580 0            0   0  8.957641   
                           5117778030860092884 0            0   0  8.956065   
                           5117778030855243220 0            0   0  8.955183   
                           5117778030907803092 0            0   0  8.901480   
                           5117778030908392916 0            0   0  8.900078   
...                                                       ...  ..       ...   
11     6142509188972423790 5117778030923072980 0            0   0  0.000000   
13     6142509188972423790 5117778030905968084 0            0   0  0.000000   
8      6142509188972423790 5117778030914553300 0            0   0  0.000000   
                           5117778030914684372 0            0   0  0.000000   
4      6142509188972423790 5117778030928905684 0            0   0  0.000000   

                                                      index size          
                                                      count  min max sum  
trange file_id             proc_id             io_cat                     
19     6142509188972423790 5117778030855898580 0          1    0   0   0  
                           5117778030860092884 0          1    0   0   0  
                           5117778030855243220 0          1    0   0   0  
                           5117778030907803092 0          1    0   0   0  
                           5117778030908392916 0          1    0   0   0  
...                                                     ...  ...  ..  ..  
11     6142509188972423790 5117778030923072980 0          1    0   0   0  
13     6142509188972423790 5117778030905968084 0          1    0   0   0  
8      6142509188972423790 5117778030914553300 0          1    0   0   0  
                           5117778030914684372 0          1    0   0   0  
4      6142509188972423790 5117778030928905684 0          2    0   0   0  

[34224 rows x 7 columns]

time: 87.5 ms (started: 2023-01-25 21:35:20 -08:00)


In [9]:
llcviews[('file_id', 'file_id', 'trange')] \
    .compute() \
    .set_index(['trange', 'file_id', 'proc_id', 'io_cat']) \
    .sort_values(('duration', 'sum'), ascending=False)


acc_pat      duration  \
                                                          min max       sum   
trange file_id             proc_id             io_cat                         
19     6142509188972423790 5117778030855898580 0            0   0  8.957641   
                           5117778030860092884 0            0   0  8.956065   
                           5117778030855243220 0            0   0  8.955183   
                           5117778030907803092 0            0   0  8.901480   
                           5117778030908392916 0            0   0  8.900078   
...                                                       ...  ..       ...   
11     6142509188972423790 5117778030895482324 0            0   0  0.000000   
                           5117778030929036756 0            0   0  0.000000   
                           5117778030896006612 0            0   0  0.000000   
8      6142509188972423790 5117778030862583252 0            0   0  0.000000   
9      6142509188972423790 5117778030905574868 0            0   0  0.000000   

                                                      index size          
                                                      count  min max sum  
trange file_id             proc_id             io_cat                     
19     6142509188972423790 5117778030855898580 0          1    0   0   0  
                           5117778030860092884 0          1    0   0   0  
                           5117778030855243220 0          1    0   0   0  
                           5117778030907803092 0          1    0   0   0  
                           5117778030908392916 0          1    0   0   0  
...                                                     ...  ...  ..  ..  
11     6142509188972423790 5117778030895482324 0          1    0   0   0  
                           5117778030929036756 0          1    0   0   0  
                           5117778030896006612 0          1    0   0   0  
8      6142509188972423790 5117778030862583252 0          1    0   0   0  
9      6142509188972423790 5117778030905574868 0          1    0   0   0  

[32987 rows x 7 columns]

time: 70.6 ms (started: 2023-01-25 21:35:36 -08:00)


In [10]:
df_1 = llcviews[('trange', 'trange', 'trange')].compute().sort_values(('duration', 'sum'), ascending=False)
df_2 = llcviews[('trange', 'trange', 'file_id')].compute().sort_values(('duration', 'sum'), ascending=False)
df_3 = llcviews[('trange', 'trange', 'proc_id')].compute().sort_values(('duration', 'sum'), ascending=False)


time: 86.3 ms (started: 2023-01-25 21:35:45 -08:00)


In [11]:
llc_value_counts = dd.concat(list(llcviews.values())) \
    .groupby(['trange', 'file_id', 'proc_id', 'io_cat']) \
    .count()['index', 'count'] \
    .compute() \
    .sort_values(ascending=False) \
    .rename(('row', 'nrepeat'))

llc_value_counts


trange  file_id              proc_id              io_cat
3       6142509188972423790  5117778030848165332  0         27
21      5347710264988741854  5117778030864090580  3         27
        5347710264898281391  5117778030926546388  3         27
                                                  1         27
        5347710264898151996  5117778030883161556  3         27
                                                            ..
5       3341432783455051907  5117778030905968084  3         18
                             5117778030905902548  3         18
                             5117778030905837012  3         18
                             5117778030905771476  3         18
34      6142509188972423790  5117778030850721236  0         18
Name: (row, nrepeat), Length: 34224, dtype: int64

time: 1.06 s (started: 2023-01-25 21:35:58 -08:00)


In [12]:
df_1.merge(llc_value_counts, on=['trange', 'file_id', 'proc_id', 'io_cat'], how='left')


trange              file_id              proc_id io_cat acc_pat      \
                                                                  min max   
0         19  6142509188972423790  5117778030855898580      0       0   0   
1         19  6142509188972423790  5117778030860092884      0       0   0   
2         19  6142509188972423790  5117778030855243220      0       0   0   
3         19  6142509188972423790  5117778030907803092      0       0   0   
4         19  6142509188972423790  5117778030908392916      0       0   0   
...      ...                  ...                  ...    ...     ...  ..   
32982     11  6142509188972423790  5117778030895482324      0       0   0   
32983     11  6142509188972423790  5117778030929036756      0       0   0   
32984     11  6142509188972423790  5117778030896006612      0       0   0   
32985      8  6142509188972423790  5117778030862583252      0       0   0   
32986      9  6142509188972423790  5117778030905574868      0       0   0   

       duration index size             row  
            sum count  min max sum nrepeat  
0      8.957641     1    0   0   0      27  
1      8.956065     1    0   0   0      27  
2      8.955183     1    0   0   0      27  
3      8.901480     1    0   0   0      27  
4      8.900078     1    0   0   0      27  
...         ...   ...  ...  ..  ..     ...  
32982  0.000000     1    0   0   0      27  
32983  0.000000     1    0   0   0      27  
32984  0.000000     1    0   0   0      27  
32985  0.000000     1    0   0   0      27  
32986  0.000000     1    0   0   0      27  

[32987 rows x 12 columns]

time: 50.8 ms (started: 2023-01-25 21:36:12 -08:00)


In [13]:
df_2.merge(llc_value_counts, on=['trange', 'file_id', 'proc_id', 'io_cat'], how='left')


trange              file_id              proc_id io_cat acc_pat      \
                                                                  min max   
0         19  6142509188972423790  5117778030855898580      0       0   0   
1         19  6142509188972423790  5117778030860092884      0       0   0   
2         19  6142509188972423790  5117778030855243220      0       0   0   
3         19  6142509188972423790  5117778030907803092      0       0   0   
4         19  6142509188972423790  5117778030908392916      0       0   0   
...      ...                  ...                  ...    ...     ...  ..   
31006     11  6142509188972423790  5117778030929036756      0       0   0   
31007     11  6142509188972423790  5117778030926349780      0       0   0   
31008     11  6142509188972423790  5117778030926153172      0       0   0   
31009     11  6142509188972423790  5117778030923072980      0       0   0   
31010      4  6142509188972423790  5117778030925891028      0       0   0   

       duration index size             row  
            sum count  min max sum nrepeat  
0      8.957641     1    0   0   0      27  
1      8.956065     1    0   0   0      27  
2      8.955183     1    0   0   0      27  
3      8.901480     1    0   0   0      27  
4      8.900078     1    0   0   0      27  
...         ...   ...  ...  ..  ..     ...  
31006  0.000000     1    0   0   0      27  
31007  0.000000     1    0   0   0      27  
31008  0.000000     1    0   0   0      27  
31009  0.000000     1    0   0   0      27  
31010  0.000000     2    0   0   0      18  

[31011 rows x 12 columns]

time: 48.4 ms (started: 2023-01-25 21:36:24 -08:00)


In [14]:
df_3.merge(llc_value_counts, on=['trange', 'file_id', 'proc_id', 'io_cat'], how='left')


trange              file_id              proc_id io_cat acc_pat      \
                                                                  min max   
0         19  6142509188972423790  5117778030855898580      0       0   0   
1         19  6142509188972423790  5117778030860092884      0       0   0   
2         19  6142509188972423790  5117778030855243220      0       0   0   
3         19  6142509188972423790  5117778030907803092      0       0   0   
4         19  6142509188972423790  5117778030908392916      0       0   0   
...      ...                  ...                  ...    ...     ...  ..   
34219     11  6142509188972423790  5117778030923072980      0       0   0   
34220     13  6142509188972423790  5117778030905968084      0       0   0   
34221      8  6142509188972423790  5117778030914553300      0       0   0   
34222      8  6142509188972423790  5117778030914684372      0       0   0   
34223      4  6142509188972423790  5117778030928905684      0       0   0   

       duration index size             row  
            sum count  min max sum nrepeat  
0      8.957641     1    0   0   0      27  
1      8.956065     1    0   0   0      27  
2      8.955183     1    0   0   0      27  
3      8.901480     1    0   0   0      27  
4      8.900078     1    0   0   0      27  
...         ...   ...  ...  ..  ..     ...  
34219  0.000000     1    0   0   0      27  
34220  0.000000     1    0   0   0      27  
34221  0.000000     1    0   0   0      27  
34222  0.000000     1    0   0   0      27  
34223  0.000000     2    0   0   0      18  

[34224 rows x 12 columns]

time: 57.9 ms (started: 2023-01-25 21:36:33 -08:00)


In [18]:
stats_df

acc_pat         app        bw                          duration  \
           max min nunique       per           sum   sum_fmt       per   
io_cat                                                                   
0            0   0       1  0.000000  0.000000e+00  0.00GB/s  0.705054   
1            0   0       1  0.289382  1.032168e+09  0.96GB/s  0.040671   
2            0   0       1  0.243110  8.671224e+08  0.81GB/s  0.048413   
3            1   0       1  0.467508  1.667507e+09  1.55GB/s  0.205862   

                     file_id            ...    rank       size                \
                 sum nunique       per  ... nunique        max          mean   
io_cat                                  ...                                    
0       14246.557617       1  0.000390  ...    1280          0  0.000000e+00   
1         821.819641    1280  0.499805  ...    1280  134217728  6.627000e+07   
2         978.242554    1280  0.499805  ...    1280  134217728  6.627000e+07   
3        4159.727051    2561  1.000000  ...    1280  629145600  1.465596e+08   

                                                        xfer                   
             min       per            sum    sum_fmt max_fmt mean_fmt min_fmt  
io_cat                                                                         
0              0  0.000000              0     0.00GB    <4KB     <4KB    <4KB  
1       25165824  0.098259   848256040960   790.00GB   >64MB    ~64MB   ~64MB  
2       25165824  0.098259   848256040960   790.00GB   >64MB    ~64MB   ~64MB  
3              0  0.803483  6936372183040  6460.00GB   >64MB    >64MB    <4KB  

[4 rows x 25 columns]

time: 48 ms (started: 2023-01-12 10:18:12 -08:00)


In [8]:
last_llc_ddf = llc_dfs[('trange','file_id','proc_id')].reset_index()
last_llc_ddf.groupby(['trange']).sum().unstack()['duration', 'sum']

trange
3      244.318817
6      122.563026
7      218.194931
8      744.576843
9     1504.269165
10    1229.440063
11     474.452148
12     403.536713
13     988.241638
14     779.520020
15     856.691467
16     869.316467
17    1251.445801
18     694.920593
19     657.794373
20      54.397316
21     289.034943
22      79.647087
23     228.763153
24     401.492828
25     876.242798
26     141.930786
dtype: float64

time: 46.4 ms (started: 2023-01-12 22:23:04 -08:00)


In [9]:
from vani._recorder.analysis import format_df

last_llc_ddf_grp = last_llc_ddf.groupby(['trange']).sum().reset_index()
last_llc_ddf_grp['duration', 'gsum'] = last_llc_ddf_grp['duration', 'sum']
last_llc_ddf_grp = format_df(df=last_llc_ddf_grp, stats_df=stats_df)
last_llc_ddf_grp

acc_pat      bw                   duration                         \
       max min per  sum   sum_fmt         csp     delta         gsum   
0        0   0 NaN  0.0  0.00GB/s   42.893608  0.012091   244.318817   
1        0   0 NaN  0.0  0.00GB/s   36.032761  0.006066   122.563026   
2        0   0 NaN  0.0  0.00GB/s   26.183456  0.010798   218.194931   
3        0   0 NaN  0.0  0.00GB/s   50.353733  0.036848   744.576843   
4        0   0 NaN  0.0  0.00GB/s  122.498024  0.074445  1504.269165   
5        0   0 NaN  0.0  0.00GB/s  171.446945  0.060844  1229.440063   
6        0   0 NaN  0.0  0.00GB/s   46.989273  0.023480   474.452148   
7        0   0 NaN  0.0  0.00GB/s   34.222275  0.019971   403.536713   
8        0   0 NaN  0.0  0.00GB/s   72.218452  0.048908   988.241638   
9        0   0 NaN  0.0  0.00GB/s   53.101040  0.038578   779.520020   
10       0   0 NaN  0.0  0.00GB/s   46.520836  0.042397   856.691467   
11       0   0 NaN  0.0  0.00GB/s   33.439690  0.043022   869.316467   
12       0   0 NaN  0.0  0.00GB/s   80.329857  0.061933  1251.445801   
13       0   0 NaN  0.0  0.00GB/s   36.807590  0.034390   694.920593   
14       0   0 NaN  0.0  0.00GB/s   30.024817  0.032554   657.794373   
15       0   0 NaN  0.0  0.00GB/s    9.798716  0.002692    54.397316   
16       0   0 NaN  0.0  0.00GB/s   19.571524  0.014304   289.034943   
17       0   0 NaN  0.0  0.00GB/s    2.924685  0.003942    79.647087   
18       0   0 NaN  0.0  0.00GB/s   24.940046  0.011321   228.763153   
19       0   0 NaN  0.0  0.00GB/s   47.133759  0.019870   401.492828   
20       0   0 NaN  0.0  0.00GB/s  197.417160  0.043365   876.242798   
21       0   0 NaN  0.0  0.00GB/s   40.923687  0.007024   141.930786   

                           ... size                             trange  \
         per          sum  ...  max mean  min  per  sum sum_fmt          
0   0.012091   244.318817  ...  0.0  0.0  0.0  0.0  0.0  0.00GB      3   
1   0.006066   122.563026  ...  0.0  0.0  0.0  0.0  0.0  0.00GB      6   
2   0.010798   218.194931  ...  0.0  0.0  0.0  0.0  0.0  0.00GB      7   
3   0.036849   744.576843  ...  0.0  0.0  0.0  0.0  0.0  0.00GB      8   
4   0.074445  1504.269165  ...  0.0  0.0  0.0  0.0  0.0  0.00GB      9   
5   0.060844  1229.440063  ...  0.0  0.0  0.0  0.0  0.0  0.00GB     10   
6   0.023480   474.452148  ...  0.0  0.0  0.0  0.0  0.0  0.00GB     11   
7   0.019971   403.536713  ...  0.0  0.0  0.0  0.0  0.0  0.00GB     12   
8   0.048907   988.241638  ...  0.0  0.0  0.0  0.0  0.0  0.00GB     13   
9   0.038578   779.520020  ...  0.0  0.0  0.0  0.0  0.0  0.00GB     14   
10  0.042397   856.691467  ...  0.0  0.0  0.0  0.0  0.0  0.00GB     15   
11  0.043022   869.316467  ...  0.0  0.0  0.0  0.0  0.0  0.00GB     16   
12  0.061933  1251.445801  ...  0.0  0.0  0.0  0.0  0.0  0.00GB     17   
13  0.034391   694.920593  ...  0.0  0.0  0.0  0.0  0.0  0.00GB     18   
14  0.032554   657.794373  ...  0.0  0.0  0.0  0.0  0.0  0.00GB     19   
15  0.002692    54.397316  ...  0.0  0.0  0.0  0.0  0.0  0.00GB     20   
16  0.014304   289.034943  ...  0.0  0.0  0.0  0.0  0.0  0.00GB     21   
17  0.003942    79.647087  ...  0.0  0.0  0.0  0.0  0.0  0.00GB     22   
18  0.011321   228.763153  ...  0.0  0.0  0.0  0.0  0.0  0.00GB     23   
19  0.019870   401.492828  ...  0.0  0.0  0.0  0.0  0.0  0.00GB     24   
20  0.043365   876.242798  ...  0.0  0.0  0.0  0.0  0.0  0.00GB     25   
21  0.007024   141.930786  ...  0.0  0.0  0.0  0.0  0.0  0.00GB     26   

      xfer                   
   max_fmt mean_fmt min_fmt  
0     <4KB     <4KB    <4KB  
1     <4KB     <4KB    <4KB  
2     <4KB     <4KB    <4KB  
3     <4KB     <4KB    <4KB  
4     <4KB     <4KB    <4KB  
5     <4KB     <4KB    <4KB  
6     <4KB     <4KB    <4KB  
7     <4KB     <4KB    <4KB  
8     <4KB     <4KB    <4KB  
9     <4KB     <4KB    <4KB  
10    <4KB     <4KB    <4KB  
11    <4KB     <4KB    <4KB  
12    <4KB     <4KB    <4KB  
13    <4KB     <4KB    <4KB  
14    <4KB     <4

time: 136 ms (started: 2023-01-12 22:23:05 -08:00)


In [10]:
import itertools as it
from pathlib import PurePath


def filename(file_id):
    path = unique_filenames[file_id]['filename']
    pure_path = PurePath(path)
    # print(pure_path.parent.name)
    if pure_path.parent.name:
        return f"../{pure_path.name}"
    return pure_path.name


def io_cat_fmt(io_cat_min, io_cat_max):
    return ''.join([['-', 'r', 'w', 'm'][io_cat] for io_cat in set([io_cat_min, io_cat_max])]).rjust(3, '-')


def procname(proc_id):
    app = unique_processes[proc_id]['app']
    hostname = unique_processes[proc_id]['hostname']
    rank = str(unique_processes[proc_id]['rank'])
    return '-'.join(filter(None, [app, hostname, rank]))


def acc_pat_fmt(acc_pat_min, acc_pat_max):
    return 'seq.' if all(x == 0 for x in [acc_pat_min, acc_pat_max]) else 'rand.'


bokeh_df = last_llc_ddf.merge(last_llc_ddf_grp, on=('trange',), how='left', suffixes=(None, '_y'))
bokeh_df = bokeh_df.sort_values(by=[('duration_y', 'gsum'), ('duration', 'sum')], ascending=[False, False])
# bokeh_df[('filename',)] = bokeh_df[("filename",)].apply(filename)
bokeh_df['trange_fmt'] = bokeh_df['trange'].apply(lambda x: f"{str(x-1).rjust(5, '0')}-{str(x).rjust(5, '0')}")
bokeh_df['filename'] = bokeh_df['file_id', ''].apply(filename)
bokeh_df['procname'] = bokeh_df['proc_id', ''].apply(procname)
bokeh_df['io_cat_fmt'] = bokeh_df.apply(lambda x: io_cat_fmt(x[('io_cat', 'min')], x[('io_cat', 'max')]), axis=1)
bokeh_df['acc_pat_fmt'] = bokeh_df.apply(lambda x: acc_pat_fmt(x[('acc_pat', 'min')], x[('acc_pat', 'max')]), axis=1)
bokeh_df


trange              file_id              proc_id acc_pat       bw       \
                                                          max min  per  sum   
48        9  5347710268612693268  5117778030853932500       0   0  0.0  0.0   
52        9  5347710266789296543  5117778030853539284       0   0  0.0  0.0   
59        9  5347710267594208907  5117778030921762260       0   0  0.0  0.0   
79        9  5347710265242774557  5117778030876804564       0   0  0.0  0.0   
80        9  5347710268408415005  5117778030875887060       0   0  0.0  0.0   
...     ...                  ...                  ...     ...  ..  ...  ...   
2537     20  6142509188972423790  5117778030859568596       0   0  0.0  0.0   
2539     20  6142509188972423790  5117778030859175380       0   0  0.0  0.0   
2645     20  6142509188972423790  5117778030859240916       0   0  0.0  0.0   
2671     20  6142509188972423790  5117778030929102292       0   0  0.0  0.0   
2697     20  6142509188972423790  5117778030928053716       0   0  0.0  0.0   

                duration            ... size_y          xfer_y           \
       sum_fmt       csp     delta  ...    sum sum_fmt max_fmt mean_fmt   
48    0.00GB/s  0.021325  0.000418  ...    0.0  0.00GB    <4KB     <4KB   
52    0.00GB/s  0.022996  0.000418  ...    0.0  0.00GB    <4KB     <4KB   
59    0.00GB/s  0.025915  0.000416  ...    0.0  0.00GB    <4KB     <4KB   
79    0.00GB/s  0.034196  0.000410  ...    0.0  0.00GB    <4KB     <4KB   
80    0.00GB/s  0.034605  0.000410  ...    0.0  0.00GB    <4KB     <4KB   
...        ...       ...       ...  ...    ...     ...     ...      ...   
2537  0.00GB/s  0.589898  0.000127  ...    0.0  0.00GB    <4KB     <4KB   
2539  0.00GB/s  0.590152  0.000127  ...    0.0  0.00GB    <4KB     <4KB   
2645  0.00GB/s  0.603167  0.000119  ...    0.0  0.00GB    <4KB     <4KB   
2671  0.00GB/s  0.606242  0.000118  ...    0.0  0.00GB    <4KB     <4KB   
2697  0.00GB/s  0.609298  0.000117  ...    0.0  0.00GB    <4KB     <4KB   

               trange_fmt                               filename  \
     min_fmt                                                       
48      <4KB  00008-00009  ../test-Part00000088-of-00001280.data   
52      <4KB  00008-00009  ../test-Part00000082-of-00001280.data   
59      <4KB  00008-00009  ../test-Part00001123-of-00001280.data   
79      <4KB  00008-00009  ../test-Part00000437-of-00001280.data   
80      <4KB  00008-00009  ../test-Part00000423-of-00001280.data   
...      ...          ...                                    ...   
2537    <4KB  00019-00020                                          
2539    <4KB  00019-00020                                          
2645    <4KB  00019-00020                                          
2671    <4KB  00019-00020                                          
2697    <4KB  00019-00020                                          

                 procname io_cat_fmt acc_pat_fmt  
                                                  
48      app1-localhost-88        --m        seq.  
52      app1-localhost-82        --m        seq.  
59    app1-localhost-1123        --m        seq.  
79     app1-localhost-437        --m        seq.  
80     app1-localhost-423        --m        seq.  
...                   ...        ...         ...  
2537   app1-localhost-174        ---        seq.  
2539   app1-localhost-168        ---        seq.  
2645   app1-localhost-169        ---        seq.  
2671  app1-localhost-1235        ---        seq.  
2697  app1-localhost-1219        ---        seq.  

[3064 rows x 63 columns]

time: 739 ms (started: 2023-01-12 22:23:06 -08:00)


In [12]:
from bokeh.io import show, save, output_file
from bokeh.models import (ColumnDataSource, DataCube, GroupingInfo, NumberFormatter,
                          StringFormatter, SumAggregator, TableColumn, CustomJS)

# output_file(filename='test.html')

source = ColumnDataSource(bokeh_df.reset_index())

target = ColumnDataSource(data=dict(row_indices=[], labels=[]))

formatter = StringFormatter(font_style='bold')
duration_formatter = NumberFormatter(format='0.000%')

columns = [
    TableColumn(field='procname_', title='Time Range > File ID > Proc ID', width=80, formatter=formatter),
    TableColumn(field='duration_per', title='Duration %', width=40,  formatter=duration_formatter),
    TableColumn(field='io_cat_fmt_', title='I/O Type', width=20),
    TableColumn(field='acc_pat_fmt_', title='Acc. Pat.', width=20),
    TableColumn(field='size_sum_fmt', title='I/O Size', width=30),
    TableColumn(field='bw_sum_fmt', title='BW', width=30),
    TableColumn(field='xfer_min_fmt', title='Min. Xfer', width=30),
    TableColumn(field='xfer_max_fmt', title='Max. Xfer', width=30),
]

grouping = [
    GroupingInfo(getter='trange_fmt_', aggregators=[SumAggregator(field_='duration_per')]),
    GroupingInfo(getter='filename_', aggregators=[SumAggregator(field_='duration_per')]),
]

cube = DataCube(source=source, columns=columns, grouping=grouping, target=target, sizing_mode='stretch_width')

show(cube)


time: 355 ms (started: 2023-01-12 22:23:39 -08:00)


In [16]:
trange_agg_cut_piv = hlm_dfs['trange'].reset_index().pivot(index='trange', columns='io_cat', values=('duration', 'sum')).fillna(0).rename_axis(columns=None)
trange_agg_cut_piv.rename(columns={0: 'other', 1:'read', 2: 'write', 3: 'metadata'}, inplace=True)
trange_agg_cut_piv

,other,read,write,metadata
trange,,,,
3,244.318817,0.000000,0.000000,0.000000
5,250.769684,0.000000,9.589067,0.000000
6,34.536442,0.000000,33.112408,1090.827515
7,7.621147,0.000000,0.000000,239.647888
8,41.223419,0.000000,0.000000,773.195923
9,188.130020,0.000000,0.000000,1317.504272
10,1046.881226,0.000000,63.450752,246.491013
11,531.094055,0.000000,47.581059,31.993809
12,582.673645,0.000000,214.898727,109.850975


time: 32.2 ms (started: 2023-01-12 22:25:12 -08:00)


In [18]:
from bokeh.io import show
from bokeh.models import ColumnDataSource
from bokeh.plotting import figure
from bokeh.palettes import Category10_3, Category10_4

symbols = list(trange_agg_cut_piv.index)
impacts = list(trange_agg_cut_piv.columns)

source = ColumnDataSource(trange_agg_cut_piv)

stat_plot = figure(width=800, height=200, tools="xpan,xbox_zoom,xbox_select,reset", active_drag="xbox_select")
stat_plot.vbar_stack(impacts, width=0.7, x="trange", source=source, color=Category10_4, legend_label=impacts)

stat_plot.y_range.start = 0
stat_plot.xgrid.grid_line_color = None
stat_plot.legend.location = "top_right"
stat_plot.legend.click_policy="mute"

show(stat_plot)

time: 235 ms (started: 2023-01-12 22:25:22 -08:00)


In [21]:
import math
from bokeh.layouts import layout
from bokeh.plotting import figure, show
from bokeh.palettes import Category10
from bokeh.transform import cumsum
from bokeh.layouts import column, row
from bokeh.models.widgets import Tabs, Panel
from bokeh.models import Legend

data = stats_df.reset_index()
data['io_cat'] = ['other', 'read', 'write', 'metadata']
data['duration', 'angle'] = data['duration', 'per'] * 2*math.pi
data['duration', 'color'] = Category10[4]
data['duration', 'per_fmt'] = data['duration', 'per'].apply(lambda x: format(float(x) * 100, ".2f") + "%")
data['size', 'angle'] = data['size', 'per'] * 2*math.pi
data['size', 'color'] = Category10[4]
data['size', 'per_fmt'] = data['size', 'per'].apply(lambda x: format(float(x) * 100, ".2f") + "%")
data['index', 'angle'] = data['index', 'per'] * 2*math.pi
data['index', 'color'] = Category10[4]
data['index', 'per_fmt'] = data['index', 'per'].apply(lambda x: format(float(x) * 100, ".2f") + "%")

def pie_chart(by: str, title: str):
    p = figure(height=200, width=300, title=title, toolbar_location=None,
               tooltips=f"@io_cat_: @{by}_per_fmt")

    p.add_layout(Legend(location='center', orientation='horizontal', padding=3, spacing=6, glyph_width=14, glyph_height=14, label_height=6), 'below')

    p.wedge(x=0, y=0, radius=0.4,
            start_angle=cumsum(f'{by}_angle', include_zero=True), end_angle=cumsum(f'{by}_angle'),
            fill_color=f'{by}_color', line_color=None, legend_field='io_cat_', source=data)

    p.axis.axis_label = None
    p.axis.visible = False
    p.grid.grid_line_color = None
#     p.legend.location = "bottom_left"
#     p.legend.orientation = "horizontal"
#     p.legend.padding = 3
#     p.legend.spacing = 6
#     p.legend.glyph_width = 14
#     p.legend.glyph_height = 14
#     p.legend.label_height = 6

    return p


plot_dur = pie_chart(by='duration', title='Agg. Duration by I/O Category')
plot_ix = pie_chart(by='index', title="Total I/O Ops by I/O Category")
plot_size = pie_chart(by='size', title="Agg. Size by I/O Category")

panel_dur = Panel(child=column(stat_plot, cube), title='Time-based')
panel_ix = Panel(child=plot_ix, title='File-based')
panel_size = Panel(child=plot_size, title='Process-based')

tabs = Tabs(tabs=[panel_dur, panel_ix, panel_size])

show(row(column([plot_dur, plot_ix, plot_size]), column([tabs])))
# show(tabs)


time: 593 ms (started: 2023-01-12 22:26:53 -08:00)
